# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [1]:
# The data represent 17 marketing campaigns

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [4]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [6]:
df.isna().sum()

,0
age,0
job,0
marital,0
education,0
default,0
housing,0
loan,0
contact,0
month,0
day_of_week,0


In [7]:
# None of the features are missing values entirely, but the following features contain "unknown" classes:
# job, marital, education, default, housing, loan

# Some other important information:
# "duration" cannot be known before a call is performed and should not be included in a realistic model.
# "pdays" is numeric, but contains a 999 placeholder value for when a client has not been previosuly contacted.

# The following features are categorical and need encoding:
# job, marital, education, contact, month, day_of_week, poutcome

# The following features have 2 (or 3, including "unknown") binary features that could be converted into booleans (or 0 and 1):
# default, housing, loan, y

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [8]:
# The objective of the task is to identify individuals who will respond positively to marketing campaigns.
# Specifically, we are predicting who will subscribe to a long-term deposit if they recieve a marketing call.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [9]:
# Remove all "unkown" values. They're just missing data.
starting_size = df.size
df = df.loc[df['job'] != 'unknown']
df = df.loc[df['marital'] != 'unknown']
df = df.loc[df['education'] != 'unknown']
df = df.loc[df['default'] != 'unknown']
df = df.loc[df['housing'] != 'unknown']
df = df.loc[df['loan'] != 'unknown']
print(df.size)
print(starting_size - df.size)

640248
224700


In [10]:
# We've lost a lot, but there's plenty left. I'm gonna go with it.
# Now let's drop the "duration" column, since it's useless.
df = df.drop('duration', axis = 1)

In [11]:
# Time to encode our features. I'm going to use dummy/one-hot encoding for allt he categorical features.
# There's some argument to be made for label encoding the education column by amount of education, but it doesn't seem appropriate.
df = pd.get_dummies(df, columns = ['job', 'marital', 'education', 'contact', 'month', 'day_of_week', 'poutcome'])

In [12]:
# Now we encode the binary features as booleans.
df['default'] = df['default'].map({'no': 0, 'yes': 1})
df['housing'] = df['housing'].map({'no': 0, 'yes': 1})
df['loan'] = df['loan'].map({'no': 0, 'yes': 1})
df['y'] = df['y'].map({'no': 0, 'yes': 1})

In [13]:
# Now to deal with pdays?
df['pdays'].value_counts()

,count
pdays,
999,29178
3,381
6,363
4,102
9,53
2,53
12,50
7,50
5,43


In [14]:
# It seems like 999 is the most common value by quite a lot.
# We essentially have two data types here: contacted/not contacted AND number of days since contact.
# The trouble is figuring out the best way to represent that to the model.
# I think I'm goint to replace "999" with 0 and add a new column for contact.
def contacted_map_function(x):
    if x == 999:
        return 0
    else:
        return 1

def pdays_map_function(x):
    if x == 999:
        return 0
    else:
        return x

df['contacted'] = df['pdays'].map(contacted_map_function)
df['pdays'] = df['pdays'].map(pdays_map_function)

In [15]:
df.value_counts()

age  default  housing  loan  campaign  pdays  previous  emp.var.rate  cons.price.idx  cons.conf.idx  euribor3m  nr.employed  y  job_admin.  job_blue-collar  job_entrepreneur  job_housemaid  job_management  job_retired  job_self-employed  job_services  job_student  job_technician  job_unemployed  marital_divorced  marital_married  marital_single  education_basic.4y  education_basic.6y  education_basic.9y  education_high.school  education_illiterate  education_professional.course  education_university.degree  contact_cellular  contact_telephone  month_apr  month_aug  month_dec  month_jul  month_jun  month_mar  month_may  month_nov  month_oct  month_sep  day_of_week_fri  day_of_week_mon  day_of_week_thu  day_of_week_tue  day_of_week_wed  poutcome_failure  poutcome_nonexistent  poutcome_success  contacted
28   0        0        0     1         0      0         1.4           93.918          -42.7          4.962      5228.1       0  False       False            False             False          False           False        False              True          False        False           False           False             True             False           False               False               False               True                   False                 False                          False                        True              False              False      False      False      True       False      False      False      False      False      False      False            False            False            True             False            False             True                  False             0            7
27   0        0        0     1         0      0         1.4           93.918          -42.7          4.962      5228.1       0  True        False            False             False          False           False        False              False         False        False           False           False             False            True            False               False               False               True                   False                 False                          False                        True              False              False      False      False      True       False      False      False      False      False      False      False            False            False            True             False            False             True                  False             0            7
43   0        1        0     1         0      0         1.4           93.918          -42.7          4.962      5228.1       0  False       False            False             True           False           False        False              False         False        False           False           False             True             False           True                False               False               False                  False                 False                          False                        True              False              False      False      False      True       False      False      False      False      False      False      False            False            False            False            True             False             True                  False             0            6
28   0        1        0     1         0      0         1.4           93.918          -42.7          4.962      5228.1       0  False       False            False             False          False           False        False              True          False        False           False           False             True             False           False               False               False               True                   False                 False                          False                        True              False              False      False      False      True       False      False      False      False      False      False      False            False            False            True             False

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [16]:
# Time to split the data.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('y', axis = 1), df['y'], test_size = 0.2)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [17]:
# Let's use a simple dummy classifier as our baseline.
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy = 'most_frequent')
dummy_clf.fit(X_train, y_train)
dummy_clf_acc = dummy_clf.score(X_test, y_test)

In [18]:
print("The accuracy of the baseline model is " + str(dummy_clf_acc) + ".")

The accuracy of the baseline model is 0.8753689734339127.


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [19]:
from math import log
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import time

# We're adding in scaling unprompted because the model didn't want to converge and this seemed to help.

log_reg_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())
])

log_reg_start_time = time.time()
log_reg_pipe.fit(X_train, y_train)
log_reg_end_time = time.time()
log_reg_time = log_reg_end_time - log_reg_start_time

### Problem 9: Score the Model

What is the accuracy of your model?

In [20]:
log_reg_acc = log_reg_pipe.score(X_test, y_test)
print("The accuracy of the basic logistic regression model is " + str(log_reg_acc) + ".")

The accuracy of the basic logistic regression model is 0.8952115447687766.


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [21]:
from sklearn.neighbors import KNeighborsClassifier

knn_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', KNeighborsClassifier())
])

knn_pipe_start_time = time.time()
knn_pipe.fit(X_train, y_train)
knn_pipe_end_time = time.time()
knn_pipe_time = knn_pipe_end_time - knn_pipe_start_time


knn_pipe.score(X_test, y_test)

0.8832404066907182

In [22]:
from sklearn.tree import DecisionTreeClassifier

tree_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', DecisionTreeClassifier())
])

tree_pipe_start_time = time.time()
tree_pipe.fit(X_train, y_train)
tree_pipe_end_time = time.time()
tree_pipe_time = tree_pipe_end_time - tree_pipe_start_time

tree_pipe.score(X_test, y_test)

0.8287963266644801

In [23]:
from sklearn.svm import SVC

svm_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', SVC())
])

svm_pipe_start_time = time.time()
svm_pipe.fit(X_train, y_train)
svm_pipe_end_time = time.time()
svm_pipe_time = svm_pipe_end_time - svm_pipe_start_time


svm_pipe.score(X_test, y_test)

0.8916038045260741

In [24]:
# Create table of model, time, train accuracy, and test accuracy.

models = ['Logistic Regression', 'KNN', 'Decision Tree', 'SVM']
times = [log_reg_time, knn_pipe_time, tree_pipe_time, svm_pipe_time]
train_accs = [
    log_reg_pipe.score(X_train, y_train),
    knn_pipe.score(X_train, y_train),
    tree_pipe.score(X_train, y_train),
    svm_pipe.score(X_train, y_train)
    ]
test_accs = [
    log_reg_pipe.score(X_test, y_test),
    knn_pipe.score(X_test, y_test),
    tree_pipe.score(X_test, y_test),
    svm_pipe.score(X_test, y_test)
    ]


model_df = pd.DataFrame({
    'Model': models,
    'Train Time': times,
    'Train Accuracy': train_accs,
    'Test Accuracy': test_accs
})

model_df

,Model,Train Time,Train Accuracy,Test Accuracy
0,Logistic Regression,0.904835,0.888356,0.895212
1,KNN,0.071037,0.902460,0.883240
2,Decision Tree,0.219286,0.993727,0.828796
3,SVM,39.103333,0.901476,0.891604


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [25]:
# Let's view the coefficient weights in a table.
coef_weight_table = pd.DataFrame({
    'Feature': X_train.columns,
    'Weight (Absolute Value)': abs(log_reg_pipe['model'].coef_[0])
})

coef_weight_table.sort_values('Weight (Absolute Value)', ascending = False)

,Feature,Weight (Absolute Value)
7,emp.var.rate,2.474738
8,cons.price.idx,1.275442
11,nr.employed,0.601836
10,euribor3m,0.307042
36,month_aug,0.219764
40,month_mar,0.216320
53,contacted,0.209280
39,month_jun,0.192318
33,contact_cellular,0.178951
34,contact_telephone,0.178951


In [26]:
# We won't be dropping gender, seeing as it isn't one of the features,
# but it may be worth dropping some of the less important features, like marital status and job.

In [27]:
# Another option is regularization. SVM's got that built-in, but it's also way, waaay slower than the other models and doesn't do much better than logistic regression.
# There's also the option of k-fold cross validation instead of an 80-20 train-test split.

In [28]:
# Let's try redoing all these models with some changes.

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures

In [30]:
X = df.drop('y', axis = 1)
y = df['y']

In [31]:
log_reg_2_params = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear', 'newton-cholesky'] # Some of these solvers don't like L1 penalties, but it's okay.
}

# I'd have tried adding some polynomial features, but it's very slow and doesn't converge. Too many features.
log_reg_2_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', GridSearchCV(LogisticRegression(max_iter=200), log_reg_2_params, cv = 5))
])

In [32]:
log_reg_2_pipe_start_time = time.time()
log_reg_2_pipe.fit(X, y)
log_reg_2_pipe_end_time = time.time()
log_reg_2_time = log_reg_2_pipe_end_time - log_reg_2_pipe_start_time

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
50 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py", line 1193, in fit
    solver = _check_solve

In [33]:
# Now we can get the accuracy score in a way that matches the accuracies above.
scaler = log_reg_2_pipe.named_steps['scaler']
X_scaled = scaler.transform(X_test)
log_reg_2_acc = log_reg_2_pipe.named_steps['model'].best_estimator_.score(X_scaled, y_test)
log_reg_2_acc

0.8919317809117744

In [34]:
# K Nearest Neighbors
knn_2_params = {
    'n_neighbors': [5, 10, 20],
    'weights': ['uniform', 'distance'],
}

knn_2_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', GridSearchCV(KNeighborsClassifier(), knn_2_params, cv = 5))
])

In [35]:
knn_2_pipe_start_time = time.time()
knn_2_pipe.fit(X, y)
knn_2_pipe_end_time = time.time()
knn_2_time = knn_2_pipe_end_time - knn_2_pipe_start_time

In [36]:
scaler = knn_2_pipe.named_steps['scaler']
X_scaled = scaler.transform(X_test)
knn_2_acc = knn_2_pipe.named_steps['model'].best_estimator_.score(X_scaled, y_test)
knn_2_acc

0.8934076746474254

In [37]:
# Decision Tree
# I won't bother to tune min_samples_split since the result would be similar to max_depth, I think.
tree_2_params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [5, 10, 20]
}

tree_2_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', GridSearchCV(DecisionTreeClassifier(), tree_2_params, cv = 5))
])

In [38]:
tree_2_pipe_start_time = time.time()
tree_2_pipe.fit(X, y)
tree_2_pipe_end_time = time.time()
tree_2_time = tree_2_pipe_end_time - tree_2_pipe_start_time

In [39]:
scaler = tree_2_pipe.named_steps['scaler']
X_scaled = scaler.transform(X_test)
tree_2_acc = tree_2_pipe.named_steps['model'].best_estimator_.score(X_scaled, y_test)
tree_2_acc

0.8955395211544769

In [42]:
# SVC (This is gonna take decades...)
svm_2_params = {
    'C': [0.01, 1, 10],
    'kernel': ['rbf'],
    'gamma': ['auto']
}

svm_2_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', GridSearchCV(SVC(), param_grid=svm_2_params, cv=2))
])

In [43]:
svm_2_start_time = time.time()
svm_2_pipe.fit(X, y)
svm_2_end_time = time.time()
svm_2_time = svm_2_end_time - svm_2_start_time

In [44]:
scaler = svm_2_pipe.named_steps['scaler']
X_scaled = scaler.transform(X_test)
svm_2_acc = svm_2_pipe.named_steps['model'].best_estimator_.score(X_scaled, y_test)
svm_2_acc

0.8753689734339127

In [47]:
# Create table of model, time, train accuracy, and test accuracy, just like above.
models_2 = ['Logistic Regression', 'KNN', 'Decision Tree', 'SVM']
times_2 = [log_reg_2_time, knn_2_time, tree_2_time, svm_2_time]
test_accs_2 = [
    log_reg_2_acc,
    knn_2_acc,
    tree_2_acc,
    svm_2_acc
    ]
score_2 = [
    log_reg_2_pipe.named_steps['model'].best_score_,
    knn_2_pipe.named_steps['model'].best_score_,
    tree_2_pipe.named_steps['model'].best_score_,
    svm_2_pipe.named_steps['model'].best_score_
    ]


model_df = pd.DataFrame({
    'Model': models_2,
    'Train Time': times_2,
    'Original Test Set Accuracy': test_accs_2,
    'Score': score_2
})

model_df

,Model,Train Time,Original Test Set Accuracy,Score
0,Logistic Regression,332.278727,0.891932,0.765799
1,KNN,40.103122,0.893408,0.697944
2,Decision Tree,3.289413,0.895540,0.405092
3,SVM,78.596278,0.875369,0.685319


In [ ]:
# It's worth noting that I was able to try out more variables for the models that can train more quickly.
# I would say that my favorite speed vs. accuracy model in this case is KNN, since it trains quickly and got the second-best result.

# Note that "Train Time" refers to the time it took for GridSearchCV to look through ALL possibilities given to it,
# not just the time it took to train the best estimator.

# ("Original Test Set Accuracy" is not very indicative of model quality and is entirely there for fun.)

##### Questions